In [ ]:
! pip install --upgrade openai

In [ ]:
! pip show openai

In [ ]:
from openai.lib.azure import AzureOpenAI
import base64
import fitz
import os
api_base = ""
api_key= ""
deployment_name = ""
api_version = ""

client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

path = "C:/Users/laksh/Downloads/Instruction Format[1].pdf"

In [ ]:
def convert_pdf_to_images(pdf_path, output_folder):
    image_name = f"{os.path.basename(pdf_path)}.png"
    image_path = os.path.join(output_folder, image_name)
    image_base64 = []
    with fitz.open(pdf_path) as doc:
        page = doc[0]
        pix = page.get_pixmap()
        pix.save(image_path)
    image_base64.append(base64.b64encode(open(image_path, "rb").read()).decode('utf-8'))
    os.remove(image_path)
    return image_base64
image_base64 = convert_pdf_to_images(path, "./")

In [ ]:
image_base64

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Get me the details of the picture in json format"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_base64[0]}"
                }
            }
        ] } 
    ],
    max_tokens=500,
    response_format= { "type":"json_object" }
)

In [ ]:
res = response.choices[0].message.content
import json 
res = json.loads(res)
res